In [0]:
from keras.preprocessing import sequence
from keras.datasets import imdb

max_words= 20000
max_len = 80
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_words)
X_train = sequence.pad_sequences(X_train,maxlen=max_len,padding='pre', truncating='pre')
X_test = sequence.pad_sequences(X_test,maxlen=max_len)

In [0]:
X_train[1:2,:]

In [0]:
y_train[1:2]

In [0]:
import numpy as np
n_classes = len(np.unique(y_train)) # n_classes = 2
y_train = np.eye(n_classes)[y_train]
y_test = np.eye(n_classes)[y_test]

In [0]:
y_train[:1]

In [0]:
y_train[1:2]

In [0]:
import tensorflow as tf
X = tf.placeholder(tf.int32,[None,80])
y = tf.placeholder(tf.int32)

In [0]:
rnn_size = 32
embedding_size = 128
W = tf.Variable(tf.truncated_normal([rnn_size,2],stddev=0.1))
B = tf.Variable(tf.truncated_normal([2],stddev=0.1))
embedding = tf.Variable(tf.truncated_normal([20000,embedding_size],stddev=0.1))

In [0]:
# Step 2 Model

from tensorflow.contrib import rnn

x_embedded = tf.nn.embedding_lookup(embedding,X)
x_embedded = tf.unstack(x_embedded,axis=1)
cell = tf.nn.rnn_cell.LSTMCell(rnn_size)
H,C = rnn.static_rnn(cell,x_embedded,dtype=tf.float32)

Ylogits= tf.matmul(H[-1],W)+B
yhat = tf.nn.softmax(Ylogits)

In [0]:
# Step 3 Loss Function

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=Ylogits))

In [0]:
# Step 4 Optimizer

lr = 0.01
train = tf.train.AdamOptimizer(lr).minimize(loss)

In [0]:
is_correct = tf.equal(tf.argmax(y,1),tf.argmax(yhat,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

training_epochs = 5
batch_size = 100
# Step 5: Training Loop
for epoch in range(training_epochs):
    for i in range(int(X_train.shape[0] / batch_size)):
        batch_X = X_train[(i*batch_size):((i+1)*batch_size)]
        batch_y = y_train[(i*batch_size):((i+1)*batch_size)]
        train_data = {X: batch_X, y: batch_y}
        sess.run(train, feed_dict=train_data)
        print(epoch, "/", i,"Training Accuracy = ", sess.run(accuracy, feed_dict=train_data))

In [0]:
acc = []
for i in range(int(X_test.shape[0] / batch_size)):
    batch_X = X_test[(i*batch_size):((i+1)*batch_size)]
    batch_y = y_test[(i*batch_size):((i+1)*batch_size)]
    test_data = {X: batch_X, y: batch_y}
    sess.run(train, feed_dict = test_data)
    acc.append(sess.run(accuracy, feed_dict = test_data))

In [0]:
acc